# 🧪 Weak Value Simulation — Pure and Mixed States

This notebook verifies two key identities from the paper:

1. For pure states:
$$\langle \phi | A P_\psi A | \phi \rangle = |\langle \phi | A | \psi \rangle|^2$$
2. For mixed states:
$$\langle \phi | A \rho A | \phi \rangle = \sum_k p_k |\langle \phi | A | \psi_k \rangle|^2$$

Both are simulated using Qiskit.

In [ ]:
!pip install qiskit qiskit-aer --quiet

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, Operator, DensityMatrix
np.random.seed(42)

## 🔬 Pure State Case: $B = A P_\psi A$

In [ ]:
A = Operator([[1, 0], [0, -1]])
psi_circuit = QuantumCircuit(1)
psi_circuit.h(0)
psi_state = Statevector(psi_circuit)
theta_phi = np.pi / 3
phi_circuit = QuantumCircuit(1)
phi_circuit.ry(theta_phi, 0)
phi_state = Statevector(phi_circuit)

In [ ]:
P_psi = Operator(np.outer(psi_state.data, psi_state.data.conj()))
B = A @ P_psi @ A
exp_B_phi = phi_state.expectation_value(B).real
amp_sq = np.abs(phi_state.data.conj() @ A.data @ psi_state.data)**2
print("Theoretical ⟨φ|B|φ⟩ =", round(exp_B_phi, 6))
print("|⟨φ|A|ψ⟩|²           =", round(amp_sq, 6))
assert np.isclose(exp_B_phi, amp_sq)

In [ ]:
phi_dagger_circuit = phi_circuit.inverse()
circuit = QuantumCircuit(1, 1)
circuit.append(psi_circuit, [0])
circuit.append(A, [0])
circuit.append(phi_dagger_circuit, [0])
circuit.measure(0, 0)
simulator = AerSimulator()
result = simulator.run(transpile(circuit, simulator), shots=10000).result()
prob_0 = result.get_counts().get('0', 0) / 10000
print("Simulated |⟨φ|A|ψ⟩|² =", round(prob_0, 6))

## 🔁 Mixed State Case: $\rho = 0.75 |0⟩⟨0| + 0.25 |1⟩⟨1|$

In [ ]:
p0, p1 = 0.75, 0.25
rho = DensityMatrix([[p0, 0], [0, p1]])
psi_k = [Statevector.from_label('0'), Statevector.from_label('1')]
pk = [p0, p1]
B_rho = Operator(A.data @ rho.data @ A.data)
exp_B_rho_phi = phi_state.expectation_value(B_rho).real
rhs_sum = sum(pk[i] * np.abs(phi_state.data.conj() @ A.data @ psi_k[i].data)**2 for i in range(2))
print("Theoretical ⟨φ|B_ρ|φ⟩ =", round(exp_B_rho_phi, 6))
print("Weighted sum         =", round(rhs_sum, 6))
assert np.isclose(exp_B_rho_phi, rhs_sum)

In [ ]:
exp_results = []
for i in range(2):
    psi_k_circuit = QuantumCircuit(1)
    if i == 1:
        psi_k_circuit.x(0)
    circuit = QuantumCircuit(1, 1)
    circuit.append(psi_k_circuit, [0])
    circuit.append(A, [0])
    circuit.append(phi_circuit.inverse(), [0])
    circuit.measure(0, 0)
    result = simulator.run(transpile(circuit, simulator), shots=10000).result()
    prob_0 = result.get_counts().get('0', 0) / 10000
    exp_results.append(prob_0)
simulated_rhs = pk[0] * exp_results[0] + pk[1] * exp_results[1]
print("Simulated RHS (mixed) =", round(simulated_rhs, 6))